## Image Classification with Transfer Learning
### This notebook trains a CNN-based classifier to distinguish between cats and dogs.

**Dataset:**  
The dataset has been imported from Kaggle, the cats and dogs dataset

**Approach:**  
We use an object-oriented approach with TensorFlow/Keras and transfer learning 
(using EfficientNetB0) to achieve at least 90% accuracy.

**Deliverables:**
- Trained model saved as `../models/cnn_model.h5`
- This notebook (`image_classification.ipynb`)

In [ ]:
pip install tensorflow

: 

In [2]:
# Import the required libraries
import os
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Check TensorFlow version
print("TensorFlow version:", tf.__version__)


ImportError: Traceback (most recent call last):
  File "c:\Users\ZAWADI\anaconda3\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [ ]:

# %% [markdown]
# ## Define the ImageClassifier Class
# 
# The `ImageClassifier` class encapsulates:
# - Loading and preprocessing the data.
# - Building the transfer learning model.
# - Training, evaluating, and saving the model.
# 
# The final model is saved to the `models` folder.

# %% [code]
class ImageClassifier:
    def __init__(self, data_dir, img_size=(224, 224), batch_size=32, val_split=0.2, seed=42):
        self.data_dir = data_dir
        self.img_size = img_size
        self.batch_size = batch_size
        self.val_split = val_split
        self.seed = seed
        self.train_ds = None
        self.val_ds = None
        self.model = None

    def load_data(self):
        """
        Loads the dataset from the data directory, splitting into training and validation sets.
        Expects subfolders in data_dir for each class.
        """
        self.train_ds = tf.keras.preprocessing.image_dataset_from_directory(
            self.data_dir,
            validation_split=self.val_split,
            subset="training",
            seed=self.seed,
            image_size=self.img_size,
            batch_size=self.batch_size
        )
        self.val_ds = tf.keras.preprocessing.image_dataset_from_directory(
            self.data_dir,
            validation_split=self.val_split,
            subset="validation",
            seed=self.seed,
            image_size=self.img_size,
            batch_size=self.batch_size
        )
        # Prefetch for performance
        AUTOTUNE = tf.data.AUTOTUNE
        self.train_ds = self.train_ds.prefetch(buffer_size=AUTOTUNE)
        self.val_ds = self.val_ds.prefetch(buffer_size=AUTOTUNE)
        print("Data loaded successfully!")
        return self.train_ds, self.val_ds

    def build_model(self, fine_tune_at=100):
        """
        Build a transfer learning model using EfficientNetB0.
        - Freeze the base model initially.
        - Add a global average pooling layer and a dense classifier.
        - Optionally, unfreeze part of the base model for fine-tuning.
        """
        base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(*self.img_size, 3))
        base_model.trainable = False  # Freeze the base model

        # Add classification head
        inputs = tf.keras.Input(shape=(*self.img_size, 3))
        x = layers.experimental.preprocessing.Rescaling(1./255)(inputs)
        x = base_model(x, training=False)
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Dropout(0.2)(x)
        outputs = layers.Dense(1, activation="sigmoid")(x)
        self.model = tf.keras.Model(inputs, outputs)

        self.model.compile(
            optimizer=optimizers.Adam(),
            loss="binary_crossentropy",
            metrics=["accuracy"]
        )
        print("Model built successfully!")
        return self.model

    def fine_tune_model(self, fine_tune_at=100):
        """
        Unfreeze the base model from the fine_tune_at layer onward and recompile for fine-tuning.
        """
        base_model = self.model.layers[2]  # EfficientNetB0 is the 3rd layer in our model
        base_model.trainable = True
        for layer in base_model.layers[:fine_tune_at]:
            layer.trainable = False
        
        self.model.compile(
            optimizer=optimizers.Adam(1e-5),  # lower learning rate for fine-tuning
            loss="binary_crossentropy",
            metrics=["accuracy"]
        )
        print("Model fine-tuning configuration updated!")
        return self.model

    def train_model(self, epochs=10):
        """
        Train the model with early stopping and model checkpoint callbacks.
        Aim for at least 90% validation accuracy.
        """
        # Ensure the models folder exists
        os.makedirs("models", exist_ok=True)
        callbacks = [
            EarlyStopping(monitor="val_accuracy", patience=3, restore_best_weights=True),
            ModelCheckpoint("models/cnn_model.h5", monitor="val_accuracy", save_best_only=True)
        ]
        history = self.model.fit(
            self.train_ds,
            validation_data=self.val_ds,
            epochs=epochs,
            callbacks=callbacks
        )
        print("Training complete!")
        return history

    def evaluate_model(self):
        """
        Evaluate the model on the validation set.
        """
        loss, accuracy = self.model.evaluate(self.val_ds)
        print(f"Validation Loss: {loss:.4f}")
        print(f"Validation Accuracy: {accuracy:.4f}")
        return loss, accuracy

    def save_model(self, filepath="models/cnn_model.h5"):
        """
        Save the trained model to the specified filepath.
        """
        self.model.save(filepath)
        print(f"Model saved as {filepath}")

# %% [markdown]
# ## Training the Model
# 
# We will now:
# 1. Load the data from `data/pet_images`
# 2. Build the transfer learning model
# 3. Train and fine-tune the model until we achieve at least 90% accuracy
# 4. Save the trained model in the `models` folder as `models/cnn_model.h5`

# Usage
# Set the dataset directory
data_dir = '../data/pet_images'

# Initialize the classifier
classifier = ImageClassifier(data_dir=data_dir, img_size=(224, 224), batch_size=32)

# Load data
classifier.load_data()

# Build the model
classifier.build_model()

# Train the model (initial training with frozen base)
history = classifier.train_model(epochs=10)

# Evaluate the model
loss, accuracy = classifier.evaluate_model()

# If validation accuracy is below 90%, fine-tune the model
if accuracy < 0.90:
    print("Validation accuracy below 90%, fine-tuning the model...")
    classifier.fine_tune_model(fine_tune_at=100)
    # Continue training
    history_ft = classifier.train_model(epochs=10)
    loss, accuracy = classifier.evaluate_model()

# Save the model in the models folder
classifier.save_model('../models/cnn_model.h5')
